# Caesar Cipher

In [4]:
def caesar_cipher(text, shift, encrypt=True, mode='alphabetical'):
    result = ""
    for char in text:
        if mode == 'alphabetical' and char.isalpha():
            shift_amount = shift if encrypt else -shift
            start = ord('A') if char.isupper() else ord('a')
            result += chr((ord(char) - start + shift_amount) % 26 + start)
        elif mode == 'ascii' and char.isprintable():
            shift_amount = shift if encrypt else -shift
            result += chr((ord(char) + shift_amount) % 128) 
        else:
            result += char
    return result

plaintext = "mario"
shift = 3

encrypted_text = caesar_cipher(plaintext, shift, encrypt=True, mode='alphabetical')
print("Encrypted (Alphabetical):", encrypted_text)

decrypted_text = caesar_cipher(encrypted_text, shift, encrypt=False, mode='alphabetical')
print("Decrypted (Alphabetical):", decrypted_text)

encrypted_text_ascii = caesar_cipher(plaintext, shift, encrypt=True, mode='ascii')
print("Encrypted (ASCII):", encrypted_text_ascii)

decrypted_text_ascii = caesar_cipher(encrypted_text_ascii, shift, encrypt=False, mode='ascii')
print("Decrypted (ASCII):", decrypted_text_ascii)


Encrypted (Alphabetical): pdulr
Decrypted (Alphabetical): mario
Encrypted (ASCII): pdulr
Decrypted (ASCII): mario


# Affine Cipher

In [1]:
def mod_inverse(a, m):
    a = a % m
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def affine_cipher(text, key_a, key_b, mod, mode='encrypt'):
    result = ""
    for char in text:
        if mod == 26:
            if char.isalpha():
                base = ord('A') if char.isupper() else ord('a')
                x = ord(char) - base
                if mode == 'encrypt':
                    y = (key_a * x + key_b) % 26
                else:
                    a_inv = mod_inverse(key_a, 26)
                    if a_inv is None:
                        raise ValueError("Key 'a' has no modular inverse under mod 26.")
                    y = (a_inv * (x - key_b)) % 26
                result += chr(y + base)
            else:
                result += char
        else:
            x = ord(char)
            if mode == 'encrypt':
                y = (key_a * x + key_b) % mod
            else:
                a_inv = mod_inverse(key_a, mod)
                if a_inv is None:
                    raise ValueError(f"Key 'a' has no modular inverse under mod {mod}.")
                y = (a_inv * (x - key_b)) % mod
            result += chr(y)
    return result

def affine_encrypt(text, key_a, key_b, mod):
    return affine_cipher(text, key_a, key_b, mod, mode='encrypt')

def affine_decrypt(text, key_a, key_b, mod):
    return affine_cipher(text, key_a, key_b, mod, mode='decrypt')

if __name__ == "__main__":
    plain_text = input("Enter the plain text: ")
    while True:
        try:
            key_a = int(input("Enter key m: "))
            break
        except ValueError:
            print("Please enter a valid integer for key a.")
    while True:
        try:
            key_b = int(input("Enter key k: "))
            break
        except ValueError:
            print("Please enter a valid integer for key b.")
    while True:
        mode_input = input("Enter the mode ('alphabet' for mod 26, 'ascii' for mod 128): ").strip().lower()
        if mode_input == 'alphabet':
            mod = 26
            break
        elif mode_input == 'ascii':
            mod = 128
            break
        else:
            print("Invalid mode. Please enter 'alphabet' or 'ascii'.")
    encrypted_text = affine_encrypt(plain_text, key_a, key_b, mod)
    decrypted_text = affine_decrypt(encrypted_text, key_a, key_b, mod)
    print("\nEncrypted text:")
    print(encrypted_text)
    print("\nDecrypted text:")
    print(decrypted_text)



Encrypted text:
hqr

Decrypted text:
adm


# Rail Fence Cipher

In [2]:
def encrypt(plaintext, depth):
    # Record space positions and remove spaces
    space_positions = [i for i, char in enumerate(plaintext) if char == ' ']
    text_without_spaces = plaintext.replace(" ", "")
    rails = [[] for _ in range(depth)]
    direction = 1
    rail = 0
    for char in text_without_spaces:
        rails[rail].append(char)
        if rail == 0:
            direction = 1
        elif rail == depth - 1:
            direction = -1
        rail += direction
    ciphertext_without_spaces = ''.join(''.join(rail) for rail in rails)
    
    # Reinsert spaces at original positions
    ciphertext_chars = list(ciphertext_without_spaces)
    for pos in space_positions:
        if pos < len(ciphertext_chars) + len(space_positions):
            ciphertext_chars.insert(pos, ' ')
    return ''.join(ciphertext_chars)

def decrypt(ciphertext, depth):
    # Record space positions and remove spaces
    space_positions = [i for i, char in enumerate(ciphertext) if char == ' ']
    text_without_spaces = ciphertext.replace(" ", "")
    length = len(text_without_spaces)
    
    # Create rail matrix and mark positions
    rails = [[''] * length for _ in range(depth)]
    direction = 1
    row = 0
    for col in range(length):
        rails[row][col] = '*'
        if row == 0:
            direction = 1
        elif row == depth - 1:
            direction = -1
        row += direction
    
    # Fill the marked positions with ciphertext characters
    index = 0
    for i in range(depth):
        for j in range(length):
            if rails[i][j] == '*' and index < length:
                rails[i][j] = text_without_spaces[index]
                index += 1
    
    # Read the matrix in zigzag order to rebuild the text
    plaintext_without_spaces = ''
    direction = 1
    row = 0
    for col in range(length):
        plaintext_without_spaces += rails[row][col]
        if row == 0:
            direction = 1
        elif row == depth - 1:
            direction = -1
        row += direction
    
    # Reinsert spaces at original positions
    plaintext_chars = list(plaintext_without_spaces)
    for pos in space_positions:
        if pos < len(plaintext_chars) + len(space_positions):
            plaintext_chars.insert(pos, ' ')
    return ''.join(plaintext_chars)

def main():
    plaintext = input("Enter the plain text message: ")
    depth = int(input("Enter the depth (key): "))
    if depth <= 1:
        print("Depth must be greater than 1")
        return
    # Ensure depth is not greater than the number of non-space characters
    non_space_count = sum(1 for char in plaintext if char != ' ')
    if depth > non_space_count:
        print("Depth cannot be greater than the number of non-space characters")
        return
    ciphertext = encrypt(plaintext, depth)
    print(f"\nEncrypted Text: {ciphertext}")
    decrypted = decrypt(ciphertext, depth)
    print(f"Decrypted Text: {decrypted}")

if __name__ == "__main__":
    main()



Encrypted Text: hloet eltx
Decrypted Text: hello text


# Hill Cipher

In [3]:
import numpy as np
import math

def mod_inverse(a, mod):    #Find the modular inverse
    a = a % mod
    for x in range(1, mod):
        if (a * x) % mod == 1:
            return x
    return None

def matrix_det(matrix, mod):     #Compute the determinant
    n = matrix.shape[0]
    if n == 2:
        return (matrix[0,0]*matrix[1,1] - matrix[0,1]*matrix[1,0]) % mod
    elif n == 3:
        return int(round(np.linalg.det(matrix)))
    else:
        raise ValueError("Only 2x2 or 3x3 matrices are supported.")

def char_to_num(c, mode):
    if mode == "a":
        return ord(c) - ord('A')  #0-25 A to Z
    elif mode == "ascii":        
        return ord(c)           

plain_text = input("Enter the plain text: ")        
key_text = input("Enter the key text: ")             
mode = input("Choose mode ('a' or 'ascii'): ").strip().lower()
if mode == "a":
    mod = 26
    plain_text = plain_text.upper()  
    key_text = key_text.upper()
elif mode == "ascii":
    mod = 128
else:
    print("Invalid mode selected. Defaulting to a.")
    mod = 26
    plain_text = plain_text.upper()
    key_text = key_text.upper()
print("Plain Text:", plain_text)
print("Key Text:", key_text)
print("Mode:", mode)
print("Modulo used:", mod)

plain_nums = [char_to_num(c, mode) for c in plain_text if c.isalnum()]   #convert PT and K to numbers 
print("\nConverting Plain Text to Numbers")
for c, num in zip(plain_text, [char_to_num(c, mode) for c in plain_text if c.isalnum()]):
    print(f"{c} -> {num}")
key_nums = [char_to_num(c, mode) for c in key_text if c.isalnum()]
print("\nConverting Key Text to Numbers")
for c, num in zip(key_text, [char_to_num(c, mode) for c in key_text if c.isalnum()]):
    print(f"{c} -> {num}")


n = int(math.sqrt(len(key_nums)))  #Form the key matrix
if n * n != len(key_nums):
    raise ValueError("Key length must be a perfect square (e.g., 4 for 2x2, 9 for 3x3, etc.).")
key_matrix = np.array(key_nums).reshape(n, n)
print(f"\nKey Matrix (dimension {n}x{n}):")
print(key_matrix)
pad_value = char_to_num('X', mode) if mode == "a" else char_to_num(' ', mode)   #Form the PT matrix
while len(plain_nums) % n != 0:
    plain_nums.append(pad_value)
print("\nPlain text numeric list:", plain_nums)
plain_vectors = []
for i in range(0, len(plain_nums), n):
    vec = np.array(plain_nums[i:i+n]).reshape(n, 1)
    plain_vectors.append(vec)
    print(f"{vec}")

#Encryption
print("\nEncryption Calculations")
encrypted_text = ""
for index, vec in enumerate(plain_vectors):
    print(f"\nEncrypting vector {index+1}:")
    print("Plain vector:\n", vec)
    product = np.dot(key_matrix, vec)
    print("Product (before modulo):\n", product)
    enc_vec = product % mod
    print("Encrypted vector (after modulo):\n", enc_vec)
    
    for num in enc_vec:
        ch = chr(int(num) + ord('A')) if mode == "a" else chr(int(num))
        encrypted_text += ch
        print(f"Converted {int(num)} -> {ch}")

print("\nFinal Encrypted Text")
print(encrypted_text)


print("\nFind the inverse of the matrix")
print("\nThe original matrix:")
print(key_matrix)

if n == 2:    #inverse of 2x2 key matrix
    raw_det = key_matrix[0, 0] * key_matrix[1, 1] - key_matrix[0, 1] * key_matrix[1, 0]
    print(f"1-Determinant of the key matrix = {raw_det}")
    det = raw_det % mod
    print(f"2-Determinant of the key matrix modulo {mod} = {det}")
    det_inv = mod_inverse(det, mod)
    if det_inv is None:
        print("Key matrix is not invertible modulo", mod)
        exit(1)
    print("3-Modular Inverse of the Determinant =", det_inv)
    a, b, c, d = key_matrix[0,0], key_matrix[0,1], key_matrix[1,0], key_matrix[1,1]
    adjugate = np.array([[d, -b], [-c, a]])
    print("4-Adjugate matrix:\n", adjugate)
    adjugate_mod = adjugate % mod
    print("5-Adjugate matrix modulo {0}:\n{1}".format(mod, adjugate_mod))
    mult_before_mod = det_inv * adjugate_mod
    print("6-Multiplication of det_inv and adjugate_mod:\n", mult_before_mod)
    inv_key_matrix = mult_before_mod % mod
    print("7-Final Inverse Key Matrix(after mod):\n", inv_key_matrix)

elif n == 3:    #inverse of 3x3 key matrix
    det = int(round(np.linalg.det(key_matrix)))
    print("1-Determinant of the key matrix =", det)
    det_mod = det % mod
    print(f"2-Determinant of the key matrix modulo {mod} = {det_mod}")
    det_inv = mod_inverse(det_mod, mod)
    if det_inv is None:
        print("Key matrix is not invertible modulo", mod)
        exit(1)
    print("3-Modular Inverse of the Determinant =", det_inv)
    minor_matrix = np.zeros((3, 3), dtype=int)
    for i in range(3):
        for j in range(3):
            minor = np.delete(np.delete(key_matrix, i, axis=0), j, axis=1)
            minor_matrix[i, j] = int(round(np.linalg.det(minor)))
    print("4(1) Adjugate of matrix :\n", minor_matrix)
    cofactor_matrix = np.zeros((3, 3), dtype=int)
    for i in range(3):
        for j in range(3):
            cofactor_matrix[i, j] = ((-1) ** (i + j)) * minor_matrix[i, j]
    print("4(2) Apply the sign :\n", cofactor_matrix)
    adjugate = cofactor_matrix.T
    print("4(3) Transpose :\n", adjugate)
    adjugate_mod = adjugate % mod
    print(f"5: Adjugate matrix modulo {mod}:\n", adjugate_mod)
    mult_before_mod = det_inv * adjugate_mod
    print(f"6: Multiplication of det_inv and adjugate_mod :\n", mult_before_mod)
    inv_key_matrix = mult_before_mod % mod
    print("7: Final Inverse Key Matrix (after mod):\n", inv_key_matrix)
else:
    print("Matrix inversion is not implemented for matrices larger than 3x3.")
    exit(1)

#Decryption
print("\nConverting Encrypted Text to Numeric Vectors for Decryption")
cipher_nums = [char_to_num(c, mode) for c in encrypted_text]
print("Cipher text numeric list:", cipher_nums)
cipher_vectors = []
for i in range(0, len(cipher_nums), n):
    vec = np.array(cipher_nums[i:i+n]).reshape(n, 1)
    cipher_vectors.append(vec)
    print(f"{vec}")
print("\nDecryption Calculations")
decrypted_text = ""
for index, vec in enumerate(cipher_vectors):
    print(f"\nDecrypting vector {index+1}:")
    print("Cipher vector:\n", vec)
    product = np.dot(inv_key_matrix, vec)
    print("Product with inverse key (before modulo):\n", product)
    dec_vec = product % mod
    print("Decrypted vector (after modulo):\n", dec_vec)
    for num in dec_vec:
        ch = chr(int(num) + ord('A')) if mode == "a" else chr(int(num))
        decrypted_text += ch
        print(f"Converted {int(num)} -> {ch}")
print("\nFinal Decrypted Text")
print(decrypted_text)

Plain Text: ASD
Key Text: QVHTVXMTG
Mode: a
Modulo used: 26

Converting Plain Text to Numbers
A -> 0
S -> 18
D -> 3

Converting Key Text to Numbers
Q -> 16
V -> 21
H -> 7
T -> 19
V -> 21
X -> 23
M -> 12
T -> 19
G -> 6

Key Matrix (dimension 3x3):
[[16 21  7]
 [19 21 23]
 [12 19  6]]

Plain text numeric list: [0, 18, 3]
[[ 0]
 [18]
 [ 3]]

Encryption Calculations

Encrypting vector 1:
Plain vector:
 [[ 0]
 [18]
 [ 3]]
Product (before modulo):
 [[399]
 [447]
 [360]]
Encrypted vector (after modulo):
 [[ 9]
 [ 5]
 [22]]
Converted 9 -> J
Converted 5 -> F
Converted 22 -> W

Final Encrypted Text
JFW

Find the inverse of the matrix

The original matrix:
[[16 21  7]
 [19 21 23]
 [12 19  6]]
1-Determinant of the key matrix = -811
2-Determinant of the key matrix modulo 26 = 21
3-Modular Inverse of the Determinant = 5
4(1) Adjugate of matrix :
 [[-311 -162  109]
 [  -7   12   52]
 [ 336  235  -63]]
4(2) Apply the sign :
 [[-311  162  109]
 [   7   12  -52]
 [ 336 -235  -63]]
4(3) Transpose :
 [[-3

C:\Users\Kero\AppData\Local\Temp\ipykernel_3436\3179563392.py:83: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ch = chr(int(num) + ord('A')) if mode == "a" else chr(int(num))
C:\Users\Kero\AppData\Local\Temp\ipykernel_3436\3179563392.py:85: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Converted {int(num)} -> {ch}")
C:\Users\Kero\AppData\Local\Temp\ipykernel_3436\3179563392.py:167: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ch = chr(int(num) + ord('A'))